<a href="https://colab.research.google.com/github/nww09001/nww09001/blob/main/SP500_Dip_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

     |████████████████████████████████| 6.3MB 4.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23455 sha256=377d756827f613a8dd38c03de573b138d164399589ebd6ad5cd7c01978eca3ec
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
!pip install yahoofinancials

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-cp37-none-any.whl size=15192 sha256=88492f613f2eb4db10b8efb3115ddad5a5cd84a4662fe7231e71df98571ce587
  Stored in directory: /root/.cache/pip/wheels/d9/7e/cf/4977a8572d5247242a4b13018d1d36923024ba84236e0d28bc
Successfully built yahoofinancials


In [3]:
!pip install requests beautifulsoup4

In [4]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
from bs4 import BeautifulSoup
import requests

In [5]:
#Extract all tickers from wiki list of S&P 500 tickers
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
table = soup.find_all('table')
#The first table is the table of all tickers
wiki_tckr = table[0]
#establish empty table
ticker_table = []
#get all table rows
rows = wiki_tckr.find_all('tr')
#For loop through table rows and extract data to table
for i in rows:
    data = i.find_all_next('td')[0].text
    data = data.replace('\n','')
    ticker_table.append(data)





In [16]:
#pull data from yfinance
daily_data = yf.download(ticker_table, start="2021-06-08", end="2021-06-09", rounding=2)

#calculate the percent change in stock price
Open = daily_data['Open']
Close = daily_data['Close']
difference = Open-Close
percent_dif = ((Close - Open)/Open)*100

df = pd.DataFrame(percent_dif)
#If statement to remove exxtra first row if needed
###Consider moving this if statement before download to reduce runtime
nrows = df.shape[0]
if nrows > 1:
  df = df.iloc[1:,:]
  print("Removed extra day")
else:
  print("No extra day to remove")
df

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
Removed extra day


,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,...,V,VFC,VIAC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-08,0.108225,-0.819001,2.327145,0.110585,-0.301739,-0.832702,-0.999692,-2.352941,-0.195035,-0.381493,-0.960903,0.36507,0.887739,-1.180893,-0.877193,-3.309638,-0.275699,0.367905,1.063224,0.98233,0.446429,0.008526,2.749562,1.319406,0.454545,0.425341,0.600709,-0.220339,-3.192247,-0.082713,-1.19702,-0.204394,-0.883104,0.144196,-1.184949,1.287776,0.544218,-0.399141,0.895937,-0.487805,...,-0.043076,0.885292,-0.023987,1.043689,0.578749,-0.140365,0.970989,-0.273486,0.204082,0.349895,0.453956,-0.157233,0.024272,0.849018,0.278552,-1.879456,-1.668628,1.060391,-0.278134,-0.914349,-0.705983,-0.321911,-0.070947,-0.483951,0.463331,0.918703,-1.667163,1.095779,-0.670204,-1.310212,-1.672145,-0.855698,1.02439,0.566182,0.782631,0.362472,-0.843404,0.033637,0.837404,-1.017682


In [29]:
#Transpose data to make it easier to work with
df = df.transpose()
#change Column name from the data to percent change
df.columns = ['Percent_Change']
df

,Percent_Change
A,0.108225
AAL,-0.819001
AAP,2.327145
AAPL,0.110585
ABBV,-0.301739
...,...
YUM,0.362472
ZBH,-0.843404
ZBRA,0.033637
ZION,0.837404


In [30]:
#sort table by percent drop
Top_drop = df.sort_values(df.columns[0])
#rename column to percent
#Top_drop.columns[0] = ['Percent']
Top_drop


,Percent_Change
MU,-5.225374
OGN,-4.681833
DISCK,-4.012241
TER,-3.830691
DISCA,-3.779628
...,...
NWS,2.973523
GNRC,3.358869
NRG,3.920438
BF.B,NaN


In [31]:
Top_tickers = Top_drop[Top_drop.Percent_Change< -5.00]
Top_tickers

,Percent_Change
MU,-5.225374


In [ ]:
#Now take these tickers and see what the price was the next day